In [1]:
# General
import os
from random import randint

# Weights & Biases
import wandb
from pytorch_lightning.loggers import WandbLogger

# Pytorch modules
import torch
from torch.nn import Linear, CrossEntropyLoss, functional as F
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split, Dataset

# Pytorch-Lightning
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, Callback
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchmetrics # a new pakage for torchmetrics
from torchmetrics.functional import accuracy

# pytorch dependent packages
import timm # try using maxvit from torchvision it should be just as good if not better

# sci-kit learn and scikit-image
import sklearn
import skimage

# Dataset
from torchvision.datasets import MNIST ######### not required
from torchvision import transforms

# to describe model
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

# number of CPUs
# cpu_count = 0 if torch.cuda.is_available() else os.cpu_count()

# use GPU tensor cores
torch.set_float32_matmul_precision('high')

In [2]:
# declaring the path of the train and test folders
train_path = "DATASET_C/TRAIN"
test_path = "DATASET_C/TEST"
classes_dir_data = os.listdir(train_path)
num_of_classes = len(classes_dir_data)
print("Total Number of Classes :" , num_of_classes)
num = 0
classes_dict = {}
classes_lst = []
num_dict = {}
for c in  classes_dir_data:
    classes_dict[c] = num
    num_dict[num] = c
    classes_lst.append(c)
    num = num +1
"""
num_dict contains a dictionary of the classes numerically and it's corresponding classes.
classes_dict contains a dictionary of the classes and the coresponding values numerically.
"""
num_of_classes = len(classes_dir_data)

classes_dict

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [3]:
#creating the dataset
#dataset
class Image_Dataset(Dataset):

    def __init__(self,classes,image_base_dir,transform = None, target_transform = None):
        """
        classes:The classes in the dataset
        image_base_dir:The directory of the folders containing the images
        transform:The trasformations for the Images
        Target_transform:The trasformations for the target
        """
        self.img_labels = classes
        self.imge_base_dir = image_base_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self,idx):
        img_dir_list = os.listdir(os.path.join(self.imge_base_dir,self.img_labels[idx]))
        image_path = img_dir_list[randint(0,len(img_dir_list)-1)]
        #print(image_path)
        image_path = os.path.join(self.imge_base_dir,self.img_labels[idx],image_path)
        image = skimage.io.imread(image_path)
        if self.transform:
            image = self.transform(image)
        if self.transform:
            label = self.target_transform(self.img_labels[idx])
        return image,label

In [4]:
size = 50 # need to be the same as what is used in layer_5/ input layer ot the cnn

basic_transformations = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((size,size)),
        transforms.Grayscale(1),
    transforms.ToTensor()])
training_transformations = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((size,size)),
    transforms.RandomRotation(degrees = 45),
    transforms.RandomHorizontalFlip(p = 0.005),
        transforms.Grayscale(1),
    transforms.ToTensor()
])

def target_transformations(x):
    return torch.tensor(classes_dict.get(x))

In [5]:
class DataModule(LightningDataModule):

    def __init__(self):
            super().__init__()            

    def prepare_data(self):
        self.train = Image_Dataset(classes_dir_data,train_path,training_transformations,target_transformations)
        self.valid = Image_Dataset(classes_dir_data,test_path,basic_transformations,target_transformations)
        self.test = Image_Dataset(classes_dir_data,test_path,basic_transformations,target_transformations)

    def train_dataloader(self):
        return DataLoader(self.train,batch_size = 64,shuffle = True)#False, num_workers = cpu_count)

    def val_dataloader(self):  
        return DataLoader(self.valid,batch_size = 64,shuffle = True)#False, num_workers = cpu_count)

    def test_dataloader(self):
        return DataLoader(self.test,batch_size = 64,shuffle = True)#False, num_workers = cpu_count)

In [6]:
class MyModel(LightningModule):

    def __init__(self, n_classes=10, acc_task="multiclass", lr=1e-3):
        super().__init__()
        
        
        """
        The convolutions are arranged in such a way that the image maintain the x and y dimensions. only the channels change
        """
        self.layer_1 = nn.Conv2d(in_channels = 1,out_channels = 3,kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.layer_2 = nn.Conv2d(in_channels = 3,out_channels = 6,kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.layer_3 = nn.Conv2d(in_channels = 6,out_channels = 12,kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.pool = nn.MaxPool2d(kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.layer_5 = nn.Linear(12*50*50,1000)#the input dimensions are (Number of dimensions * height * width)
        self.layer_6 = nn.Linear(1000,100)
        self.layer_7 = nn.Linear(100,50)
        self.layer_8 = nn.Linear(50,10)
        self.layer_9 = nn.Linear(10,10)
        
        
        
        # metrics
        self.acc_task = acc_task
        self.lr = lr
        self.n_classes = n_classes
        self.accuracy = torchmetrics.Accuracy(task=self.acc_task, num_classes=self.n_classes)
        self.class_names = classes_lst
        self.loss = CrossEntropyLoss()

        # optional - save hyper-parameters to self.hparams
        # they will also be automatically logged as config parameters in W&B
        self.save_hyperparameters()

    def forward(self,x):
        """
        x is the input data
        """
        x = self.layer_1(x)
        x = self.pool(x)
        x = self.layer_2(x)
        x = self.pool(x)
        x = self.layer_3(x)
        x = self.pool(x)
        x = x.view(x.size(0),-1)
        print(x.size())
        x = self.layer_5(x)
        x = self.layer_6(x)
        x = self.layer_7(x)
        x = self.layer_8(x)
        x = self.layer_9(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr = self.lr)
        return optimizer

# The Pytorch-Lightning module handles all the iterations of the epoch

    def training_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('train_loss', loss)
        # Log metrics
        self.log('train_acc', self.accuracy(y_pred, y))
        return loss

    def validation_step(self,batch,batch_idx):
        preds, loss, acc = self._get_preds_loss_accuracy(batch)
        # Log loss and metric
        self.log('val_loss_alt', loss)
        self.log('val_accuracy_alt', acc)
        
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('val_loss', loss)
        # Log metrics
        self.log('val_acc', self.accuracy(y_pred, y))
        self.cpu_pred = y_pred.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"val_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_pred,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})
        return preds

    def test_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('test_loss', loss)
        # Log metrics
        self.log('test_acc', self.accuracy(y_pred, y))
        self.cpu_pred = y_pred.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"test_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_pred,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})
        return loss
    
    def _get_preds_loss_accuracy(self, batch):
        '''convenience function since train/valid/test steps are similar'''
        x, y = batch
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        loss = self.loss(logits, y)
        acc = accuracy(preds, y, self.acc_task, num_classes=10)
        return preds, loss, acc

In [7]:
checkpoint_callback = ModelCheckpoint(monitor='val_acc', mode='max')

class LogPredictionsCallback(Callback):
    
    def on_validation_batch_end(
        self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        """Called when the validation batch ends."""
 
        # `outputs` comes from `LightningModule.validation_step`
        # which corresponds to our model predictions in this case
        
        # Let's log 20 sample image predictions from first batch
        if batch_idx == 0:
            n = 20
            x, y = batch
            images = [img for img in x[:n]]
            captions = [f'Ground Truth: {y_i} - Prediction: {y_pred}' for y_i, y_pred in zip(y[:n], outputs[:n])]
            
            # Option 1: log images with `WandbLogger.log_image`
            wandb_logger.log_image(key='sample_images', images=images, caption=captions)

            # Option 2: log predictions as a Table
            columns = ['image', 'ground truth', 'prediction']
            data = [[wandb.Image(x_i), y_i, y_pred] for x_i, y_i, y_pred in list(zip(x[:n], y[:n], outputs[:n]))]
            wandb_logger.log_table(key='sample_table', columns=columns, data=data)

log_predictions_callback = LogPredictionsCallback()

In [8]:
wandb.login()
wandb_logger = WandbLogger(project='computer_vision_test_single', log_model=True)

# TRAIN
# setup data
# data = MNISTDataModule()
data = DataModule()

# setup model - choose different hyperparameters per experiment
model = MyModel(n_classes=num_of_classes)


trainer = Trainer(
    accelerator='gpu', 
    devices=-1, # use all GPU's (-1)
    #callbacks=[log_predictions_callback, checkpoint_callbacks], # log_predictions_callback, checkpoint_callback
    logger=wandb_logger,    # W&B integration
    max_epochs=5            # number of epochs
    )

In [9]:
trainer.fit(model, data)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [10]:
trainer.test(model, datamodule=data)

[{'test_loss': 2.639571189880371, 'test_acc': 0.10000000149011612}]

Testing: 0it [00:00, ?it/s]

In [11]:
wandb_logger.experiment

In [12]:
wandb_logger.experiment.finish()